In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error
from joblib import dump, load
import scipy as sp

In [2]:
df = pd.read_csv("Advertising.csv")

## Sımple Linear Regression

In [ ]:
df.head()

In [ ]:
df["total_spend"] = df["TV"] + df["radio"] + df["newspaper"]
df.head()

In [ ]:
plt.figure(figsize=(12,7), dpi=120)
sns.regplot(data=df, x="total_spend", y="sales")

In [ ]:
x = df["total_spend"]
y = df["sales"]

In [ ]:
np.polyfit(x,y,deg=1)

In [ ]:
potential_spend=np.linspace(0,500,100)
predicted_sales = 0.04868788*potential_spend + 4.24302822

sns.scatterplot(x="total_spend", y="sales", data=df)
plt.plot(potential_spend,predicted_sales, color="red")

In [ ]:
spend=200
predicted_sales = 0.04868788*spend + 4.24302822
predicted_sales

## Scikit-Learn

In [ ]:
df = pd.read_csv("Advertising.csv")
df.head()

In [ ]:
fig,axes = plt.subplots(nrows=1,ncols=3,figsize=(16,6))

axes[0].plot(df['TV'],df['sales'],'o')
axes[0].set_ylabel("Sales")
axes[0].set_title("TV Spend")

axes[1].plot(df['radio'],df['sales'],'o')
axes[1].set_title("Radio Spend")
axes[1].set_ylabel("Sales")

axes[2].plot(df['newspaper'],df['sales'],'o')
axes[2].set_title("Newspaper Spend");
axes[2].set_ylabel("Sales")
plt.tight_layout();

In [ ]:
sns.pairplot(df)

In [ ]:
x= df.drop("sales", axis=1)
y= df["sales"]
print(x)
print(y)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x,
y, random_state=101, test_size=0.3)

In [ ]:
print(len(df))
print(len(x_train))
print(len(x_test))

In [ ]:
model= LinearRegression()

In [ ]:
model.fit(x_train, y_train)

In [ ]:
model.predict(x_test)

In [ ]:
x_test.head()

In [ ]:
y_test.head()

In [ ]:
test_predictions = model.predict(x_test)

In [ ]:
print(df["sales"].mean())
sns.histplot(data=df, x="sales", bins=20);

In [ ]:
mean_absolute_error(y_test, test_predictions)
# Mean value ile kıyaslanmalı

In [ ]:
np.sqrt(mean_squared_error(y_test, test_predictions))
#root mean square error

In [ ]:
test_residuals = y_test - test_predictions
test_residuals.head()

In [ ]:
sns.scatterplot(x=y_test, y=test_residuals)
plt.axhline(y= 0, color="red", ls="--")

In [ ]:
sns.displot(test_residuals,bins=25, kde=True)

In [ ]:
# Create a figure and axis to plot on
fig, ax = plt.subplots(figsize=(6,4),dpi=100)
# probplot returns the raw values if needed
# we just want to see the plot, so we assign these values to _
_ = sp.stats.probplot(test_residuals,plot=ax)

In [ ]:
final_model = LinearRegression()
final_model.fit(x,y)
print(final_model.coef_)

In [ ]:
x.head()

In [ ]:
y_hat = final_model.predict(x)

In [ ]:
fig,axes = plt.subplots(nrows=1,ncols=3,figsize=(16,6))

axes[0].plot(df['TV'],df['sales'],'o')
axes[0].plot(df['TV'],y_hat,'o',color='red')
axes[0].set_ylabel("Sales")
axes[0].set_title("TV Spend")

axes[1].plot(df['radio'],df['sales'],'o')
axes[1].plot(df['radio'],y_hat,'o',color='red')
axes[1].set_title("Radio Spend")
axes[1].set_ylabel("Sales")

axes[2].plot(df['newspaper'],df['sales'],'o')
axes[2].plot(df['radio'],y_hat,'o',color='red')
axes[2].set_title("Newspaper Spend");
axes[2].set_ylabel("Sales")
plt.tight_layout();

In [ ]:
dump(final_model, 'sales_model.joblib') 

In [ ]:
loaded_model = load('sales_model.joblib')

In [ ]:
loaded_model.coef_

In [ ]:
campaign = [[149,22,12]]

In [ ]:
loaded_model.predict(campaign)

## Polynomial Regression

In [ ]:
x = df.drop("sales", axis=1)
y = df["sales"]
print(x.head())
print(y.head())

In [ ]:
polynomial_converter = PolynomialFeatures(degree= 2, include_bias= False)

In [ ]:
polynomial_converter.fit(x)

In [ ]:
poly_features = polynomial_converter.fit_transform(x)

In [ ]:
print(polynomial_converter.transform(x).shape)
print(x.shape)

In [ ]:
print(x.iloc[0])
print(poly_features[0])

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(poly_features,
 y, test_size=0.3, random_state=101)

In [ ]:
model = LinearRegression()
model.fit(x_train, y_train)

In [ ]:
test_predictions = model.predict(x_test)

In [ ]:
MAE = mean_absolute_error(y_test,test_predictions)
MSE = mean_squared_error(y_test,test_predictions)
RMSE = np.sqrt(MSE)

In [ ]:
print(f"MAE = {round(MAE,3)}")
print(f"MSE = {round(MSE,3)}")
print(f"RMSE = {round(RMSE,3)}")

## Choosing Model

In [ ]:
# TRAINING ERROR PER DEGREE
train_rmse_errors = []
# TEST ERROR PER DEGREE
test_rmse_errors = []

for d in range(1,10):
    
    # CREATE POLY DATA SET FOR DEGREE "d"
    polynomial_converter = PolynomialFeatures(degree=d,include_bias=False)
    poly_features = polynomial_converter.fit_transform(x)
    
    # SPLIT THIS NEW POLY DATA SET
    x_train, x_test, y_train, y_test = train_test_split(poly_features,
     y, test_size=0.3, random_state=101)
    
    # TRAIN ON THIS NEW POLY SET
    model = LinearRegression(fit_intercept=True)
    model.fit(x_train,y_train)
    
    # PREDICT ON BOTH TRAIN AND TEST
    train_pred = model.predict(x_train)
    test_pred = model.predict(x_test)
    
    # Calculate Errors
    
    # Errors on Train Set
    train_RMSE = np.sqrt(mean_squared_error(y_train,train_pred))
    
    # Errors on Test Set
    test_RMSE = np.sqrt(mean_squared_error(y_test,test_pred))

    # Append errors to lists for plotting later
   
    train_rmse_errors.append(train_RMSE)
    test_rmse_errors.append(test_RMSE)

In [ ]:
train_rmse_errors

In [ ]:
test_rmse_errors

In [ ]:
plt.plot(range(1,6),train_rmse_errors[:5],label='TRAIN')
plt.plot(range(1,6),test_rmse_errors[:5],label='TEST')
plt.xlabel("Polynomial Complexity")
plt.ylabel("RMSE")
plt.legend()

In [ ]:
plt.plot(range(1,10),train_rmse_errors,label='TRAIN')
plt.plot(range(1,10),test_rmse_errors,label='TEST')
plt.xlabel("Polynomial Complexity")
plt.ylabel("RMSE")
plt.legend()

In [ ]:
plt.plot(range(1,10),train_rmse_errors,label='TRAIN')
plt.plot(range(1,10),test_rmse_errors,label='TEST')
plt.xlabel("Polynomial Complexity")
plt.ylabel("RMSE")
plt.ylim(0,100)
plt.legend()

## Final Model Evaluation

In [ ]:
# Based on our chart, could have also been degree=4, but 
# it is better to be on the safe side of complexity
final_poly_converter = PolynomialFeatures(degree=3,include_bias=False)
final_model = LinearRegression()
final_model.fit(final_poly_converter.fit_transform(x),y)

In [ ]:
from joblib import dump, load
dump(final_model, 'sales_poly_model.joblib') 
dump(final_poly_converter,'poly_converter.joblib')

In [ ]:
loaded_poly = load('poly_converter.joblib')
loaded_model = load('sales_poly_model.joblib')
campaign = [[149,22,12]]
campaign_poly = loaded_poly.transform(campaign)

In [ ]:
campaign_poly

In [ ]:
final_model.predict(campaign_poly)

## Regularization

In [5]:
df = pd.read_csv("Advertising.csv")
X = df.drop("sales", axis=1)
y = df["sales"]
print(X)
print(y)

        TV  radio  newspaper
0    230.1   37.8       69.2
1     44.5   39.3       45.1
2     17.2   45.9       69.3
3    151.5   41.3       58.5
4    180.8   10.8       58.4
..     ...    ...        ...
195   38.2    3.7       13.8
196   94.2    4.9        8.1
197  177.0    9.3        6.4
198  283.6   42.0       66.2
199  232.1    8.6        8.7

[200 rows x 3 columns]
0      22.1
1      10.4
2       9.3
3      18.5
4      12.9
       ... 
195     7.6
196     9.7
197    12.8
198    25.5
199    13.4
Name: sales, Length: 200, dtype: float64


In [6]:
polynomial_converter = PolynomialFeatures(degree=3, include_bias=False)

In [7]:
poly_features = polynomial_converter.fit_transform(X)

In [8]:
X_train, X_test, y_train, y_test = train_test_split(poly_features,
 y, test_size=0.3, random_state=101)

In [9]:
scaler = StandardScaler()
scaler.fit(X_train)

StandardScaler()

In [10]:
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

## Ridge Regression

In [13]:
from sklearn.linear_model import Ridge

In [14]:
ridge_model = Ridge(alpha=10)
ridge_model.fit(X_train,y_train)

Ridge(alpha=10)

In [15]:
test_predictions = ridge_model.predict(X_test)

In [16]:
MAE = mean_absolute_error(y_test,test_predictions)
MSE = mean_squared_error(y_test,test_predictions)
RMSE = np.sqrt(MSE)
print(f"MAE = {round(MAE,3)}")
print(f"MSE = {round(MSE,3)}")
print(f"RMSE = {round(RMSE,3)}")

MAE = 0.577
MSE = 0.8
RMSE = 0.895


In [17]:
from sklearn.linear_model import RidgeCV

In [25]:
ridge_cv_model = RidgeCV(alphas = (0.1, 1, 10), scoring="neg_mean_absolute_error")

In [26]:
ridge_cv_model.fit(X_train, y_train)

RidgeCV(alphas=array([ 0.1,  1. , 10. ]), scoring='neg_mean_absolute_error')

In [24]:
from sklearn.metrics import SCORERS

In [27]:
test_predictions = ridge_cv_model.predict(X_test)

In [28]:
MAE = mean_absolute_error(y_test,test_predictions)
MSE = mean_squared_error(y_test,test_predictions)
RMSE = np.sqrt(MSE)
print(f"MAE = {round(MAE,3)}")
print(f"MSE = {round(MSE,3)}")
print(f"RMSE = {round(RMSE,3)}")

MAE = 0.427
MSE = 0.382
RMSE = 0.618


## Lasso Regression

In [29]:
from sklearn.linear_model import LassoCV

In [32]:
Lasso_cv_model = LassoCV(eps=0.1, n_alphas=100, cv=5)

In [33]:
Lasso_cv_model.fit(X_train, y_train)

LassoCV(cv=5, eps=0.1)

In [34]:
test_predictions = Lasso_cv_model.predict(X_test)

In [35]:
MAE = mean_absolute_error(y_test,test_predictions)
MSE = mean_squared_error(y_test,test_predictions)
RMSE = np.sqrt(MSE)
print(f"MAE = {round(MAE,3)}")
print(f"MSE = {round(MSE,3)}")
print(f"RMSE = {round(RMSE,3)}")

MAE = 0.654
MSE = 1.279
RMSE = 1.131


## Elastic Net

In [36]:
from sklearn.linear_model import ElasticNetCV

In [37]:
elastic_model = ElasticNetCV(l1_ratio=[.1, .5, .7,.9, .95, .99, 1],tol=0.01)

In [38]:
elastic_model.fit(X_train,y_train)

ElasticNetCV(l1_ratio=[0.1, 0.5, 0.7, 0.9, 0.95, 0.99, 1], tol=0.01)

In [39]:
test_predictions = elastic_model.predict(X_test)

In [40]:
MAE = mean_absolute_error(y_test,test_predictions)
MSE = mean_squared_error(y_test,test_predictions)
RMSE = np.sqrt(MSE)
print(f"MAE = {round(MAE,3)}")
print(f"MSE = {round(MSE,3)}")
print(f"RMSE = {round(RMSE,3)}")

MAE = 0.566
MSE = 0.56
RMSE = 0.749


In [41]:
train_predictions = elastic_model.predict(X_train)
MAE = mean_absolute_error(y_train,train_predictions)
MAE

0.4307582990472369